In [1]:
import pandas as pd
import numpy as np
import h5py as h
from sklearn.metrics import roc_curve, roc_auc_score
from scipy.stats import norm
from matplotlib import pyplot
from tqdm import trange
import feather

/opt/apps/rhel7/Python-3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Gather variables

In [102]:
enrichr_dataset = ['go_bp_', 'mgi_', 'kegg_']
curr = enrichr_dataset[2]

In [103]:
f = h.File("auc_data.hdf5", "r+")
data = f['data']
meta = f['meta']

In [104]:
gslib = data['tcga_' + curr + 'gslib']
tcga_genes = [str(g[0])[2:-1] for g in meta['tcga_genes']]
curr_genes = [str(g[0])[2:-1] for g in meta[curr + 'genes']]
curr_pheno = [str(g[0])[2:-1] for g in meta[curr + 'pheno']]
curr_gslib = data[curr + 'gslib']
binary_matrix = data[curr + "bin_mat"]

In [ ]:
df = feather.read_dataframe("human_correlation")

# Calculate new gene set libraries

In [105]:
cor = np.corrcoef(binary_matrix)

In [106]:
np.fill_diagonal(cor, None)

In [107]:
cor = pd.DataFrame(cor)

In [108]:
pheno_to_gene = {}
# to get indices associated with a particular phenotype
rev = np.transpose(binary_matrix)
for i in range(len(curr_pheno)): 
    pheno_to_gene[i] = np.where(rev[i] == 1)[0]

In [109]:
predictions = pd.DataFrame()
count = []
preds = []
for j in trange(len(curr_pheno)):
    indices = pheno_to_gene[j]
    preds.append(cor.iloc[:, indices].mean(axis=1))
predictions = pd.concat(preds, axis=1)
predictions = predictions.fillna(0)

100%|██████████| 308/308 [00:05<00:00, 58.77it/s]


In [13]:
predictions

,0,1,2,3,4,5,6,7,8,9,...,5251,5252,5253,5254,5255,5256,5257,5258,5259,5260
0,0.166419,0.070845,0.086941,0.026899,0.020440,0.170768,0.044923,0.033181,0.044535,0.013362,...,0.001357,-0.001318,0.034827,0.007187,0.001053,0.009609,0.018196,0.027469,0.031237,0.025190
1,0.102684,0.050884,0.038441,0.013548,0.019360,0.093394,0.029923,0.020935,0.024445,0.036797,...,0.009215,0.000249,0.014099,0.023604,0.020589,0.028562,0.016972,0.061871,0.059801,0.043040
2,0.086942,0.046166,0.044643,0.031645,0.038215,0.077924,0.045911,0.047398,0.040056,0.019863,...,0.014406,0.004386,0.039292,0.020961,0.005136,0.034919,0.016089,0.044383,0.049331,0.034492
3,0.116485,0.099034,0.120243,0.051805,0.028684,0.059238,0.065890,0.037115,0.044451,0.011930,...,0.021725,0.054115,0.004092,0.050656,0.048177,0.044152,0.068729,0.003894,0.028136,-0.002276
4,0.074979,0.025079,0.019545,0.012447,0.018192,0.085755,0.015717,0.019571,0.028295,0.008127,...,0.007121,0.001121,0.020639,0.010841,0.006802,0.006269,0.013368,0.094376,0.070322,0.106975
5,0.113224,0.094917,0.119822,0.051435,0.031223,0.051484,0.051393,0.043245,0.042528,0.013122,...,0.009591,0.021275,0.000613,0.026016,0.008066,0.018949,0.033274,0.005756,0.007344,0.003722
6,0.039319,0.012231,0.011192,0.017598,0.012628,0.009119,0.029016,0.062840,-0.001497,0.007090,...,-0.002601,-0.006012,0.015639,0.000661,-0.006737,0.012498,-0.000905,0.011717,-0.004195,0.004158
7,0.140798,0.208933,0.151365,0.055611,0.029568,0.069566,0.065444,0.032186,0.048266,0.016337,...,-0.003994,0.008297,0.008585,0.014802,0.003060,0.003458,0.015546,0.015673,0.044975,0.008051
8,0.068793,0.024250,0.018118,0.027348,0.022035,0.051303,0.023818,0.027203,0.040899,0.011753,...,0.008717,-0.009466,0.002338,0.028744,-0.001840,0.008640,0.005291,0.019807,0.022005,0.020991
9,0.073312,0.040695,0.064395,0.035814,0.021797,0.036757,0.039320,0.028536,0.038441,0.012625,...,0.003113,0.005838,-0.001093,0.008609,-0.001424,0.031536,0.006831,0.011129,0.008389,0.007057


In [110]:
"""
Expanded mouse gene set library with the same number of genes as the TCGA gene set.
We should ignore the ~3400 mouse genes not found in the TCGA gene set since they won't 
be included in the above calculations.
"""

ex_mgsl = np.zeros((len(archs4_genes), len(curr_pheno)))

In [111]:
# TCGA gene to index dictionary to help fill in expanded mouse gene set library 
tcga_to_idx = {} 
for i in range(len(archs4_genes)): 
    g = archs4_genes[i]
    tcga_to_idx[g] = i

In [112]:
"""
Loop through the current mouse gene names. If the mouse gene name is found in the tcga_to_idx
dictionary, we find its index according to the TCGA gene list and replace the ex_mgsl row of 
zeros with the row found in the previous mouse gene set library. All of the genes found in the
TCGA library but not in the mouse gene set library will be left as zero for phenotype correlations.
"""
for m in range(len(curr_genes)): 
    curr_gene = curr_genes[m]
    if curr_gene in tcga_to_idx:
        idx = tcga_to_idx[curr_gene]
        ex_mgsl[idx] = predictions.iloc[m]  # replace expanded mgsl row with the prev mgsl row of correlations

In [113]:
ex_mgsl.shape

(26415, 308)

In [120]:
ex_mgsl

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.00449915, -0.00175986, -0.01212325, ..., -0.00592863,
        -0.01301921, -0.00518748],
       ...,
       [-0.00449915,  0.05996596,  0.00407872, ...,  0.00153604,
         0.01388089,  0.01161785],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [83]:
tcga = h.File("tcga.hdf5", "r")

In [84]:
tcga_cor = tcga['full correlation matrix']

In [114]:
"""
We can compute the numerator part of the matrix by multiplying matrices together.
Use Numpy rather than go through matrix manually b/c np probably has some speedy
magic we don't know about.
"""
# tcga correlation matrix and ex_mgsl
gslib = np.matmul(np.array(df), ex_mgsl)

In [115]:
# Check to get a new matrix with TCGA genes as rows and phenotypes of columns
gslib.shape

(26415, 308)

In [116]:
"""
To finish computing the gene set library we have to go through each of the entries and divide
by the sum of the correlations in that phenotype's set (minus the current gene's correlation).
We can speed up computations by just taking the sums of each phenotype column. As we loop 
through the genes for each phenotype, we can just subtract the current gene's correlation
from the phenotype's sum.
"""

pheno_sums = list(np.sum(predictions))

for i in trange(len(gslib)):
    for j in range(len(curr_pheno)):
        sub = ex_mgsl[i][j]
        denom = pheno_sums[j]
        gslib[i][j] /= (denom-sub)

100%|██████████| 26415/26415 [00:19<00:00, 1377.79it/s]


In [88]:
del data['tcga_' + curr + 'gslib']
data.create_dataset('tcga_' + curr + 'gslib', data=gslib)

<HDF5 dataset "tcga_kegg_gslib": shape (38550, 308), type "<f8">

In [89]:
del data[curr + 'gslib']
data.create_dataset(curr + 'gslib', data=predictions)

<HDF5 dataset "kegg_gslib": shape (7802, 308), type "<f8">

In [26]:
pd.DataFrame(gslib)

,0,1,2,3,4,5,6,7,8,9,...,5093,5094,5095,5096,5097,5098,5099,5100,5101,5102
0,0.000115,-0.005102,-0.000763,-0.010697,-0.004113,-0.007055,0.004507,-0.003281,-0.002088,-0.005474,...,-0.001637,-0.000264,-0.001884,-0.004452,-0.002598,-0.002790,-0.004785,0.011420,-0.002124,0.001426
1,-1.796972,-0.650158,0.322589,-3.367721,-1.321236,-1.704289,1.047635,-0.309543,-0.170954,-1.274559,...,-0.436130,-0.201470,-0.764064,-0.881648,-0.468571,0.800835,-1.654677,2.203503,-0.377153,-0.513510
2,-1.928019,0.228531,1.384957,-3.470058,-0.560066,-0.990969,0.268236,0.424631,0.088112,-0.999841,...,0.413002,-0.483956,0.531255,-0.224049,0.948750,2.055788,-2.168852,0.823705,-0.349050,-1.572236
3,0.921287,0.504707,0.372301,-0.892042,0.693560,0.026923,0.454547,0.427782,-0.060907,-0.029095,...,0.750149,0.273928,0.954588,0.468688,0.818837,0.296313,-0.870899,0.089997,-0.039279,-0.160080
4,1.509729,0.673034,0.301455,0.265197,1.162209,0.559640,0.402615,0.548371,0.019864,0.405534,...,0.534645,0.636522,1.058272,0.789698,0.852844,0.209163,-0.723167,0.167996,0.182555,0.271490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26410,3.051582,0.252745,-0.840849,2.220014,0.809569,0.852873,0.825697,-0.395492,-0.411141,0.556459,...,0.305543,0.829182,0.453678,0.535851,-0.610969,-1.425318,1.281790,0.312485,0.288988,1.951967
26411,2.139677,1.435426,1.694334,-2.390476,1.994803,0.412306,2.485573,1.176133,-0.302368,-0.034614,...,1.399205,1.005985,1.370805,1.145588,1.727245,0.010893,-2.104114,0.306501,0.432346,-0.855285
26412,-0.848791,2.249375,2.914024,-1.686320,2.784451,1.499634,-1.080815,3.278103,1.961482,1.023356,...,2.200051,0.462578,4.066361,2.095485,5.603458,5.821239,-1.741902,-0.577664,0.349899,-1.669996
26413,1.067072,1.251075,1.136559,-0.505631,2.009680,0.887967,0.444765,1.311915,0.495761,0.514182,...,1.463412,0.836109,2.156748,1.300121,2.285393,1.667824,-1.023371,-0.153020,0.142498,-0.645809


In [91]:
data.create_dataset("archs4_" + curr + "gslib", data=gslib)
f.close()

# Compare with ARCHS4 dataset

In [96]:
np.fill_diagonal(np.array(df), 0)

In [80]:
archs4_genes = list(df.columns)

# Calculate AUC

In [117]:
common_genes = list(set(archs4_genes) & set(curr_genes))

Calculate average AUC by gene of the gene set library with its binary matrix. Should approximate 1

In [95]:
auc_list = []
for g in range(len(curr_genes)):
    y_true = binary_matrix[g]
    y_probs = predictions.iloc[g]
    auc = roc_auc_score(y_true, y_probs)
    auc_list.append(auc)
np.mean(auc_list)

0.9991029832435975

Calculate average AUC by gene of the prediction matrix with the current dataset's binary matrix

In [118]:
auc_list = []
curr_genes_t = np.transpose(curr_genes)
tcga_genes_t = np.transpose(archs4_genes)
for g in common_genes:
    a = np.where(curr_genes_t == g)[0][0]
    b = np.where(tcga_genes_t == g)[0][0]
    y_true = binary_matrix[a]
    y_probs = gslib[b]
    auc = roc_auc_score(y_true, y_probs)
    auc_list.append(auc)
np.mean(auc_list)

0.6698193413808486

In [119]:
# TCGA gene to index dictionary to help fill in expanded mouse gene set library 
tcga_to_idx = {} 
for i in range(len(archs4_genes)): 
    g = archs4_genes[i]
    tcga_to_idx[g] = i
    
common_idx = [ tcga_to_idx[g] for g in common_genes ]
common_binary_idx = [ np.where(np.transpose(curr_genes) == g)[0][0] for g in common_genes ]
smaller_gslib = pd.DataFrame(gslib).iloc[common_idx]
smaller_binary = pd.DataFrame(binary_matrix).iloc[common_binary_idx]
pheno_auc = []
for p in trange(len(curr_pheno)):
    y_true = smaller_binary.loc[:, p]
    y_probs = smaller_gslib.loc[:, p]
    auc = roc_auc_score(y_true, y_probs)
    pheno_auc.append(auc)
np.mean(pheno_auc)

100%|██████████| 308/308 [00:01<00:00, 303.37it/s]


0.5940584901366848